In [1]:
#шаг 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#шаг 2
!pip install -q transformers datasets peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#шаг 3
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# шаг 4
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM  # ✅ Импортируем недостающие модули

# Указываем название модели
model_name = "ai-forever/rugpt3small_based_on_gpt2"

# Загружаем данные из файла
with open('/content/drive/MyDrive/clean_poems.jsonl', 'r', encoding='utf-8') as f:
    clean_poems = [json.loads(line)['messages'][1]['content'] for line in f]

# Конвертируем в датасет
dataset = Dataset.from_dict({"text": clean_poems})

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Исправляем проблему с pad_token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Функция токенизации
def tokenize_function(examples):
    encodings = tokenizer(
        examples['text'],
        padding="max_length",  # Паддинг до максимальной длины
        truncation=True,       # Обрезка длинных текстов
        max_length=256,        # Максимальная длина
        return_tensors="pt"
    )
    encodings['labels'] = encodings['input_ids'].clone()
    return encodings

# Токенизация датасета
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
train_dataset = tokenized_dataset


Map:   0%|          | 0/567 [00:00<?, ? examples/s]

In [7]:
# шаг 5

from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

# Конфигурация LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["attn.c_attn", "attn.c_proj"]  # ✅ Исправили target_modules
)

# Встраиваем LoRA в модель
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 126,116,352 || trainable%: 0.7015


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [8]:
#шаг 6
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/phi2_poetry_model',
    num_train_epochs=10,
    per_device_train_batch_size=1,  # Работает даже на слабых GPU
    save_steps=500,
    logging_steps=10,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [9]:
#шаг 7
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,8.669200
20,8.897100
30,8.635100
40,7.629700
50,7.467400
60,6.260400
70,4.851500
80,3.726400
90,2.637900
100,2.955700


TrainOutput(global_step=5670, training_loss=1.7979054017041727, metrics={'train_runtime': 322.9246, 'train_samples_per_second': 17.558, 'train_steps_per_second': 17.558, 'total_flos': 748468182712320.0, 'train_loss': 1.7979054017041727, 'epoch': 10.0})

In [50]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/poetry_model_phi2',
    num_train_epochs=2,  # Дополнительные эпохи
    per_device_train_batch_size=2,
    warmup_steps=100,
    weight_decay=0.001,
    learning_rate=1e-5,  # Уменьшаем скорость обучения
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

Step,Training Loss
10,2.210900
20,1.886000
30,1.310200
40,1.547700
50,1.966200
60,1.599300
70,1.049400
80,1.802600
90,1.730800
100,2.170200


TrainOutput(global_step=568, training_loss=1.6565522711041947, metrics={'train_runtime': 51.4175, 'train_samples_per_second': 22.055, 'train_steps_per_second': 11.047, 'total_flos': 149693636542464.0, 'train_loss': 1.6565522711041947, 'epoch': 2.0})

In [57]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/poetry_model_phi2_finetune',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    warmup_steps=50,
    weight_decay=0.01,
    learning_rate=3e-6,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

Step,Training Loss
10,2.207600
20,1.881500
30,1.304200
40,1.540700
50,1.959500
60,1.591600
70,1.043900
80,1.793700
90,1.718500
100,2.159100


TrainOutput(global_step=2840, training_loss=1.6489733148628558, metrics={'train_runtime': 218.6785, 'train_samples_per_second': 25.928, 'train_steps_per_second': 12.987, 'total_flos': 748468182712320.0, 'train_loss': 1.6489733148628558, 'epoch': 10.0})

In [51]:
#шаг 8
model.save_pretrained('/content/drive/MyDrive/final_phi2_poetry_model')
tokenizer.save_pretrained('/content/drive/MyDrive/final_phi2_tokenizer')
print("✅ Обучение завершено!")


✅ Обучение завершено!


In [64]:
# 1й вариант генерации
def generate_poem(prompt):
    # Преобразуем prompt в input_ids
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    # Генерируем стих с улучшенными параметрами
    output = model.generate(input_ids,
                            max_new_tokens=150,  # Увеличиваем количество токенов для более длинного текста
                            do_sample=True,
                            temperature=0.8,  # Температура ближе к 1 для баланса между креативностью и связностью
                            top_p=0.92,  # Ограничиваем выбор слов до наиболее вероятных
                            top_k=40,  # Уменьшаем top_k для большей предсказуемости
                            repetition_penalty=1.5,  # Штраф за повторения, чтобы избежать зацикливания
                            num_return_sequences=1,  # Генерируем только один вариант
                            no_repeat_ngram_size=2,  # Запрещаем повторение биграмм
                            early_stopping=True)  # Останавливаем генерацию, если текст завершен

    # Преобразуем вывод в текст и добавляем переносы строк после каждой точки
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    formatted_text = generated_text.replace(". ", ".\n")  # Добавляем перенос после каждой точки

    return formatted_text

# Запрашиваем у пользователя тему стиха и генерируем стих
user_prompt = input("Введите тему стиха: ")
print(generate_poem(user_prompt))


Введите тему стиха: Осень
Осень в сердце моем
Не стоит унывать и печалиться, -
Останьтесь с любимыми людьми.
Начинать надо осторожно: вдруг кто-то не так поймет...
(с)Ромашка! В этом году нам будет море улыбок )))))))))
Приходите к нам на концерт вместе со своей любимой собакой или кошкой :) А то мы сегодня тоже пошалили :( У вас такой интересный шанс встретить свою единственную ) Тогда у нас точно есть свой человек...
С вами можно общаться очень хорошо ))).
Запишись под катом ;)А когда придёт время - приходите!!! И поможете стать лучше друзьями! 
Ну а если вам скучно что ли? Ну тогда просто присоединяйтесь )))
Твои стихи уже


In [83]:
# 2й вариант генерации
def generate_poem(prompt):
    # Преобразуем prompt в input_ids
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

    # Генерируем стих с новыми параметрами
    output = model.generate(input_ids,
                            max_new_tokens=100,  # Увеличиваем количество новых токенов
                            do_sample=True,
                            temperature=0.7,  # Снижаем температуру для уменьшения случайности
                            top_p=2.25,  # Ограничиваем топ-п для более контролируемого выбора
                            top_k=40,
                            repetition_penalty=1.5  # Штраф за повторения, чтобы избежать зацикливания
                            )  # Снижаем top_k для большей предсказуемости

    # Преобразуем вывод в текст с добавлением переноса строк
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    formatted_text = generated_text.replace(". ", ".\n")  # Добавляем перенос после каждой точки

    return formatted_text

# Запрашиваем у пользователя тему стиха и генерируем стих
user_prompt = input("Введите тему стиха: ")
print(generate_poem(user_prompt))


Введите тему стиха: Марионетка
Марионетка
Твоя бабушка...
ты знаешь, кто я? )))))))
А у тебя есть друзья??:)) ))) И не надо об этом.
Я всегда о тебе хорошо отзываюсь-всегда готов выслушать и помочь!:) Заходи на огонёк - всё будет просто отлично) Всё это время буду ждать!) А вообще...
Настроение хорошее такое же как сейчас=*( а что с ним?) *___ (не знаю).
Ты наверно думаешь про себя:"я здесь в очереди стою


In [76]:
# 3й вариант генерации
def generate_poem(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(
        input_ids,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,  # Снижаем для большей предсказуемости
        top_p=0.9,  # Стандартное значение для баланса
        top_k=30,  # Уменьшаем для большей точности
        repetition_penalty=1.2,  # Меньший штраф, чтобы сохранить естественность
        no_repeat_ngram_size=3,  # Запрещаем повторение триграмм
        early_stopping=True
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    formatted_text = generated_text.replace(". ", ".\n")
    return formatted_text

user_prompt = input("Введите тему стиха: ")
print(generate_poem(user_prompt))

Введите тему стиха: Вечно молодой
Вечно молодой,
Не забывай ни о себе, ни о тебе.
И пусть все будет так как ты хочешь!

Прости меня за то что я был не прав...
Но знай: это всё - лишь сон...
Ты должен забыть обо мне и никогда больше....
Спаси тебя Господи от всех невзгод!!!
Напиши мне на прощенье хоть раз "ты" в письме,
Чтобы впредь была жизнь лучше без боли.
Я люблю тебя!!!!!!
Здравствуй


In [82]:
# 4й вариант генерации
def generate_poem(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(
        input_ids,
        max_new_tokens=120,
        do_sample=True,
        temperature=0.6,  # Ещё ниже для предсказуемости
        top_p=0.85,  # Уже диапазон
        top_k=20,  # Ещё меньше для точности
        repetition_penalty=1.2,  # Мягче штраф
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    formatted_text = generated_text.replace(". ", ".\n").replace("! ", "!\n").replace("? ", "?\n")
    return formatted_text

user_prompt = input("Введите тему стиха: ")
print(generate_poem(user_prompt))

Введите тему стиха: Вечно молодой
Вечно молодой
Париж, Париж и Ницца...
Счастливый не может быть одинок.
Но вот беда - ты уже один!
В тебе есть что-то от меня?
Ты ведь очень молод, я тебя помню:
И в то же время я был без ума,
Что было бы мне так приятно.
Так зачем жить с тобой?
Зачем всё это?
Ведь ты мой ангел,
Я знаю, как трудно любить.
Не верь мне на слово,
Ведь я лишь твой ангел.
Как тяжело порой бывает нам...
Будь рядом со мной
